Getting Started
===============

The pins package helps you publish data sets, models, and other R objects, making it easy to share them across projects and with your colleagues.
You can pin objects to a variety of "boards", including local folders (to share on a networked drive or with dropbox), RStudio connect, Amazon S3, and more.
This vignette will introduce you to the basics of pins.

In [1]:
import pins

## Getting started

Every pin lives in a pin *board*, so you must start by creating a pin board.
In this vignette I'll use a temporary board which is automatically deleted when your R session is over:

In [2]:
# TODO: simple board constructors
import fsspec
import tempfile

fs = fsspec.filesystem("file")
tmp_dir = tempfile.TemporaryDirectory()

board = pins.BaseBoard(tmp_dir.name, fs)

In real-life, you'd pick a board depending on how you want to share the data.
Here are a few options:

In [3]:
# TODO
# board <- board_local() # share data across R sessions on the same computer
# board <- board_folder("~/Dropbox") # share data with others using dropbox
# board <- board_folder("Z:\\my-team\pins") # share data using a shared network drive
# board <- board_rsconnect() # share data with RStudio Connect

## Reading and writing data

Once you have a pin board, you can write data to it with `pin_write()`:

In [4]:
from siuba.data import mtcars
board.pin_write(mtcars, "mtcars", type="csv")

Meta(title='A pinned 32 x 11 CSV', description=None, created='20220315T210341Z', pin_hash='7f10d4e9942bb928', file='mtcars', file_size=1420, type='csv', api_version=1, version=Version(created=datetime.datetime(2022, 3, 15, 21, 3, 41, 660284), hash='7f10d4e9942bb928'), name='mtcars', user={})

The first argument is the object to save (usually a data frame, but it can be any R object), and the second argument gives the "name" of the pin.
The name is basically equivalent to a file name: you'll use it when you later want to read the data from the pin.
The only rule for a pin name is that it can't contain slashes.